In [ ]:
import os
import glob
import argparse
from torch.utils.data import DataLoader, random_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from modelsli1 import *
import json
import numpy as np
from ate import*
import pandas as pd
import sklearn.metrics
from cvxopt import matrix, solvers

In [ ]:
def kernel(ker, X1, X2, gamma):
    """
    Kernel function to compute kernel matrix based on kernel type.
    :param ker: 'linear' | 'rbf'
    :param X1: First dataset (Xs or Xt)
    :param X2: Second dataset (Xs or Xt)
    :param gamma: Kernel bandwidth (only used for 'rbf')
    :return: Computed kernel matrix
    """
    K = None
    if ker == 'linear':
        if X2 is not None:
            K = sklearn.metrics.pairwise.linear_kernel(np.asarray(X1), np.asarray(X2))
        else:
            K = sklearn.metrics.pairwise.linear_kernel(np.asarray(X1))
    elif ker == 'rbf':
        if X2 is not None:
            K = sklearn.metrics.pairwise.rbf_kernel(np.asarray(X1), np.asarray(X2), gamma)
        else:
            K = sklearn.metrics.pairwise.rbf_kernel(np.asarray(X1), None, gamma)
    return K
class KMM:
    def __init__(self, kernel_type='linear', gamma=1.0, B=1.0, eps=None):
        '''
        Initialization function
        :param kernel_type: 'linear' | 'rbf'
        :param gamma: kernel bandwidth for rbf kernel
        :param B: bound for beta
        :param eps: bound for sigma_beta
        '''
        self.kernel_type = kernel_type
        self.gamma = gamma
        self.B = B
        self.eps = eps

    def fit(self, Xs, Xt):
        '''
        Fit source and target using KMM (compute the coefficients)
        :param Xs: ns * dim
        :param Xt: nt * dim
        :return: Coefficients (Pt / Ps) value vector (Beta in the paper)
        '''
        ns = Xs.shape[0]
        nt = Xt.shape[0]
        if self.eps is None:
            self.eps = self.B / np.sqrt(ns)
        
        # Compute kernel matrix
        K = kernel(self.kernel_type, Xs, None, self.gamma)
        kappa = np.sum(kernel(self.kernel_type, Xs, Xt, self.gamma) * float(ns) / float(nt), axis=1)
        
        # Set up and solve the quadratic programming problem
        K = matrix(K.astype(np.double))
        kappa = matrix(kappa.astype(np.double))
        G = matrix(np.r_[np.ones((1, ns)), -np.ones((1, ns)), np.eye(ns), -np.eye(ns)])
        h = matrix(np.r_[ns * (1 + self.eps), ns * (self.eps - 1), self.B * np.ones((ns,)), np.zeros((ns,))])

        sol = solvers.qp(K, -kappa, G, h)
        beta = np.array(sol['x'])
        return beta
def apply_kmm(Xs, Ys, Xt, Yt, kernel_type='rbf', gamma=1.0, B=1.0):
    """
    Apply KMM to source and target domain data to compute new source data.
    :param Xs: Source data (ns * dim)
    :param Ys: Source labels (ns * 1)
    :param Xt: Target data (nt * dim)
    :param Yt: Target labels (nt * 1)
    :param kernel_type: 'linear' | 'rbf', default is 'rbf'
    :param gamma: Bandwidth parameter for 'rbf' kernel, default is 1.0
    :param B: Bound for beta, default is 1.0
    :return: New source data Xs_new after applying KMM
    """
    # Initialize KMM model
    kmm = KMM(kernel_type=kernel_type, gamma=gamma, B=B)
    
    # Fit KMM model to compute the coefficients
    beta = kmm.fit(Xs, Xt)
    
    # Compute the new source data Xs_new by scaling the original Xs with beta
    Xs_new = beta * Xs
    
    return Xs_new

In [ ]:
#引入数据
def load_target(file_path='/Users/asus/Desktop/SCM/target.csv'):
    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()
        if content.startswith('\ufeff'):  # 检查并去除 BOM
            content = content[1:]

    # 使用 np.loadtxt 读取文件内容（处理了 BOM 后的内容）
    from io import StringIO
    data = np.loadtxt(StringIO(content), delimiter=',')
    x=data[:, 2:]
    t, y,  = data[:, 0], data[:, 1][:, None]
    return x,t.reshape(-1, 1),y
def load_source(file_path='/Users/asus/Desktop/SCM/source.csv'):
    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()
        if content.startswith('\ufeff'):  # 检查并去除 BOM
            content = content[1:]

    # 使用 np.loadtxt 读取文件内容（处理了 BOM 后的内容）
    from io import StringIO
    data1 = np.loadtxt(StringIO(content), delimiter=',')
    x=data1[:, 2:]
    t, y,  = data1[:, 0], data1[:, 1][:, None]
    return x,t.reshape(-1, 1),y

In [ ]:
def get_estimate(q_t0, q_t1, g, t, y_dragon, index, eps, truncate_level=0.01):
    """
    getting the back door adjustment & TMLE estimation
    """

    psi_n = psi_naive(q_t0, q_t1, g, t, y_dragon, truncate_level=truncate_level)
    ipw_n, dr_n = psi_weighting(q_t0, q_t1, g, t, y_dragon, truncate_level=truncate_level)
    psi_tmle, psi_tmle_std, eps_hat, initial_loss, final_loss, g_loss = psi_tmle_cont_outcome(q_t0, q_t1, g, t,
                                                                                              y_dragon,
                                                                                              truncate_level=truncate_level)
    return psi_n, psi_tmle, initial_loss, final_loss, g_loss,ipw_n, dr_n
class EarlyStopper:
    def __init__(self, patience=1, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.min_validation_loss = np.inf
    def early_stop(self, validation_loss):
        if validation_loss < self.min_validation_loss:
            self.min_validation_loss = validation_loss
            self.counter = 0
        elif validation_loss > (self.min_validation_loss + self.min_delta):
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False

In [ ]:
def _split_output(yt_hat, t, y, y_scaler, x, index):
    """
        Split output into dictionary for easier use in estimation#为了以后方便使用
        Args:
            yt_hat: Generated prediction，生成的预测，有两个y0与y1
            t: Binary treatment assignments
            y: Treatment outcomes,实际已有的数据
            y_scaler: Scaled treatment outcomes#标准化后的
            x: Covariates
            index: Index in data

        Returns:
            Dictionary of all needed data
    """
    yt_hat = yt_hat.detach().cpu().numpy()#将 yt_hat 从 PyTorch 的张量转换成 NumPy 数组（脱离计算图，移到 CPU）。
    q_t0 = y_scaler.inverse_transform(yt_hat[:, 0].reshape(-1, 1).copy())#归一化后的对照组潜在预测结果
    q_t1 = y_scaler.inverse_transform(yt_hat[:, 1].reshape(-1, 1).copy())
    g = yt_hat[:, 2].copy()# 提取倾向得分

    if yt_hat.shape[1] == 4:
        eps = yt_hat[:, 3][0]# 如果 `yt_hat` 有第四列，提取 `eps`
    else:
        eps = np.zeros_like(yt_hat[:, 2])
    y = y_scaler.inverse_transform(y.copy())#逆归一化
    var = "average propensity for treated: {} and untreated: {}".format(g[t.squeeze() == 1.].mean(),
                                                                        g[t.squeeze() == 0.].mean())
    print(var)

    return {'q_t0': q_t0, 'q_t1': q_t1, 'g': g, 't': t, 'y': y, 'x': x, 'index': index, 'eps': eps}

In [ ]:
def train(train_loader, net, optimizer, criterion,valid_loader= None,l1_reg = None):
    """
    Trains network for one epoch in batches.
    Args:
        train_loader: Data loader for training set.
        net: Neural network model.
        optimizer: Optimizer (e.g. SGD).优化器
        criterion: Loss function (e.g. cross-entropy loss).
    """
    avg_loss = 0

    for i, data in enumerate(train_loader):
        # get the inputs; data is a list of [inputs, labels]，获取输入;data 是 [inputs， labels] 的列表
        inputs, labels = data

        # zero the parameter gradients，参数梯度归零
        optimizer.zero_grad()

        # forward + backward + optimize，前进、反馈、最优化
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        if l1_reg is not None:
            l1_penalty = l1_reg * sum([p.abs().sum() for p in net.parameters()])
            loss+= l1_penalty
        loss.backward()
        optimizer.step()

        # keep track of loss and accuracy，跟踪损失和准确性
        avg_loss += loss

    valid_loss = None
    if valid_loader is not None:
        valid_loss = 0.0
        net.eval()     # Optional when not using Model Specific layer，不使用模型特定图层时可选
        for data, labels in valid_loader:
            if torch.cuda.is_available():
                data, labels = data.cuda(), labels.cuda()
            
            target = net(data)
            loss = criterion(target,labels)
            if l1_reg is not None:
                loss+= l1_reg * sum([p.abs().sum() for p in net.parameters()]) 
            valid_loss += loss
        valid_loss = valid_loss/len(valid_loader)
    return avg_loss / len(train_loader), valid_loss

In [ ]:
def train_and_predict_dragons(t, y_unscaled, x, net,seed = 0, targeted_regularization=True, output_dir='',
                              knob_loss=dragonnet_loss_binarycross, ratio=1., dragon='', val_split=0.2, batch_size=64,lr =1e-3,l1_reg = None):
    """
    Method for training dragonnet and tarnet and predicting new results
    Returns:
        Outputs on train and test data
用于训练 dragonnet 和 tarnet 并预测新结果的方法，
返回： train 和 test 数据的输出
    """    
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(device)

    verbose = 0
    y_scaler = StandardScaler()
    y = y_scaler.fit_transform(y_unscaled)
    train_outputs = []#定义元组，将输出存在里面
    test_outputs = []


    # Which loss to use for training the network，选择损失函数，在dragonnet_loss与普通knob_loss中选取
    if targeted_regularization:
        loss = make_tarreg_loss(ratio=ratio, dragonnet_loss=knob_loss)
    else:
        loss = knob_loss


    i = seed
    torch.manual_seed(i)
    np.random.seed(i)
    random.seed(i)

    if ratio == 0:
        train_index = np.arange(x.shape[0])
        test_index = train_index
    else:
        train_index, test_index = train_test_split(np.arange(x.shape[0]), test_size=ratio, random_state=seed)
        print(f'test_index {test_index}')
   
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    t_train, t_test = t[train_index], t[test_index]

    yt_train = np.concatenate([y_train, t_train], 1)

    yt_test = np.concatenate([y_test, t_test], 1)

    # Create data loader to pass onto training method，创建数据加载器以传递到训练方法
    tensors_train = torch.from_numpy(x_train).float().to(device), torch.from_numpy(yt_train).float().to(device)
    train_size = int((val_split) * len(TensorDataset(*tensors_train)))
    val_size = int(len(TensorDataset(*tensors_train))-train_size)
    train_set, valid_set = random_split(TensorDataset(*tensors_train),[train_size,val_size])
    train_loader = DataLoader(train_set, batch_size=batch_size)
    valid_loader = DataLoader(valid_set, batch_size=500)

    import time;
    start_time = time.time()

    # Configuring optimizers，配置优化器，惩罚迭代过程
    # Training the networks first for 100 epochs with the Adam optimizer and，首先使用 Adam 优化器训练网络 100 个 epoch
    # then for 300 epochs with the SGD optimizer.Adam 用于初始阶段，SGD 用于更长的训练阶段
    epochs1 = 100
    epochs2 = 300

    # Add L2 regularization to t0 and t1 heads of the network，将 L2 正则化添加到网络的 t0 和 t1 头
    optimizer_Adam = optim.Adam([{'params': net.representation_block.parameters()},
                                 {'params': net.t_predictions.parameters()},
                                 {'params': net.t0_head.parameters(), 'weight_decay': 0.01},
                                 {'params': net.t1_head.parameters(), 'weight_decay': 0.01}], lr=lr)
    optimizer_SGD = optim.SGD([{'params': net.representation_block.parameters()},
                               {'params': net.t_predictions.parameters()},
                               {'params': net.t0_head.parameters(), 'weight_decay': 0.01},
                               {'params': net.t1_head.parameters(), 'weight_decay': 0.01}], lr=lr*0.01, momentum=0.9)
    scheduler_Adam = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer_Adam, mode='min', factor=0.5, patience=5,
                                                          threshold=1e-8, cooldown=0, min_lr=0)
    scheduler_SGD = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer_SGD, mode='min', factor=0.5, patience=5,
                                                         threshold=0, cooldown=0, min_lr=0)

    train_loss = 0

    early_stopper = EarlyStopper(patience=2, min_delta=0.)

    # Adam training run
    for epoch in range(epochs1):

        # Train on data
        train_loss,val_loss = train(train_loader, net, optimizer_Adam, loss,valid_loader = valid_loader,l1_reg = l1_reg)
        
        if early_stopper.early_stop(val_loss):             
            break

        scheduler_Adam.step(val_loss)

    print(f"Adam loss: train -- {train_loss}, validation -- {val_loss}, epoch {epoch}")

    # SGD training run
    
    early_stopper = EarlyStopper(patience=40, min_delta=0.)

    for epoch in range(epochs2):
        # Train on data
        train_loss,val_loss = train(train_loader, net, optimizer_SGD, loss,valid_loader = valid_loader,l1_reg = l1_reg)

        if early_stopper.early_stop(val_loss):             
            break
        scheduler_SGD.step(val_loss)
        

    print(f"SGD loss: train --  {train_loss}, validation -- {val_loss},  epoch {epoch}")

    elapsed_time = time.time() - start_time
    print("***************************** elapsed_time is: ", elapsed_time)

    yt_hat_test = net(torch.from_numpy(x_test).float().to(device))
    yt_hat_train = net(torch.from_numpy(x_train).float().to(device))

    test_outputs += [_split_output(yt_hat_test, t_test, y_test, y_scaler, x_test, test_index)]
    train_outputs += [_split_output(yt_hat_train, t_train, y_train, y_scaler, x_train, train_index)]
   
    train_all_dicts = _split_output(yt_hat_train, t_train, y_train, y_scaler, x_train, train_index)
    test_all_dicts = _split_output(yt_hat_test, t_test, y_test, y_scaler, x_test, test_index)
   
    psi_n, psi_tmle, initial_loss, final_loss, g_loss,ipw_n, dr_n = get_estimate(train_all_dicts['q_t0'].reshape(-1, 1), train_all_dicts['q_t1'].reshape(-1, 1), train_all_dicts['g'].reshape(-1, 1), train_all_dicts['t'].reshape(-1, 1), train_all_dicts['y'].reshape(-1, 1), train_all_dicts['index'].reshape(-1, 1), train_all_dicts['eps'].reshape(-1, 1),truncate_level=0.01)

    train_dict = {'psi_n':psi_n, 'classification_mse': g_loss,'ipw_n':ipw_n, 'dr_n':dr_n,'regression_loss':regression_loss(torch.tensor(yt_train).to(device),yt_hat_train).cpu().detach(),'BCE':binary_classification_loss(torch.tensor(yt_train).float().to(device),yt_hat_train).cpu().detach().numpy(),'regression_mse':initial_loss,'index':train_all_dicts['index']}
    
    psi_n, psi_tmle, initial_loss, final_loss, g_loss,ipw_n, dr_n = get_estimate(test_all_dicts['q_t0'].reshape(-1, 1), test_all_dicts['q_t1'].reshape(-1, 1), test_all_dicts['g'].reshape(-1, 1), test_all_dicts['t'].reshape(-1, 1), test_all_dicts['y'].reshape(-1, 1), test_all_dicts['index'].reshape(-1, 1), test_all_dicts['eps'].reshape(-1, 1),truncate_level=0.01)

    
    test_dict = {'psi_n':psi_n, 'classification_mse': g_loss,'ipw_n':ipw_n, 'dr_n':dr_n,'regression_loss':regression_loss(torch.tensor(yt_test).to(device),yt_hat_test).cpu().detach(),'BCE':binary_classification_loss(torch.tensor(yt_test).float().to(device),yt_hat_test).cpu().detach().numpy(),'regression_mses':initial_loss,'index':test_all_dicts['index']}

    return test_outputs, train_outputs, net,train_dict,test_dict

In [ ]:
import torch

def convert_to_serializable(obj):
    """递归地将不可序列化的对象转换为 JSON 可序列化的形式"""
    if isinstance(obj, dict):
        return {k: convert_to_serializable(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [convert_to_serializable(i) for i in obj]
    elif isinstance(obj, torch.Tensor):  # 处理 PyTorch Tensor
        return obj.tolist() if obj.dim() > 0 else obj.item()
    elif isinstance(obj, np.ndarray):  # NumPy 数组转列表
        return obj.tolist()
    elif isinstance(obj, (np.float32, np.float64)):  # NumPy 浮点转标准浮点
        return float(obj)
    elif isinstance(obj, (np.int32, np.int64)):  # NumPy 整数转标准整数
        return int(obj)
    else:
        return obj

In [ ]:
def run_KMMsplit(data_base_dir='/Users/asus/Desktop/SCM', output_dir='/Users/asus/Desktop/SCM',
             knob_loss=dragonnet_loss_binarycross, ratio=1., dragon='', lr2=1e-3, l1_reg=1e-3, batchsize2=64):
    print("the dragon is {}".format(dragon))
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    x_t, t_t, y_t = load_target()
    x_s, t_s, y_s = load_source()
    # 按照二分类数据 t 列进行分类
    # 确保 t 是一维数组并且只用于行索引
    x_t0, y_t0 = x_t[t_t.ravel() == 0], y_t[t_t.ravel() == 0]#目标域的处理与对照的划分
    x_t1, y_t1 = x_t[t_t.ravel() == 1], y_t[t_t.ravel() == 1]
    x_s0, y_s0 = x_s[t_s.ravel() == 0], y_s[t_s.ravel() == 0]#源域的处理与对照的划分
    x_s1, y_s1 = x_s[t_s.ravel() == 1], y_s[t_s.ravel() == 1]
    x_s0_new = apply_kmm(x_t0, y_t0, x_s0, y_s0, kernel_type='rbf', gamma=1.0, B=1.0)
    x_s1_new = apply_kmm(x_t1, y_t1, x_s1, y_s1, kernel_type='rbf', gamma=1.0, B=1.0)

        # 合并新源域数据
    Xs_new = np.vstack((x_s0_new, x_s1_new))

    final_output = []
    for is_targeted_regularization in [False]:
        print("Is targeted regularization: {}".format(is_targeted_regularization))
        net = TarNet(x_s.shape[1]).to(device) if dragon == 'tarnet' else DragonNet(x_s.shape[1]).to(device)
        
        # Train source data
        _, _, net, _, _ = train_and_predict_dragons(t_s, y_s, Xs_new, net, seed=42,
                                                    targeted_regularization=is_targeted_regularization,
                                                    knob_loss=knob_loss, ratio=0, dragon=dragon,
                                                    val_split=0.3, batch_size=64, lr=1e-3)
        
        parm = {}
        for name, param in net.named_parameters():
            param.grad = None
            parm[name]=param.detach()

        if dragon == 'tarnet':
            print('I am here making tarnet')
            net = TarNet_transfer(x_s.shape[1],parm).to(device)

        elif dragon == 'dragonnet':
            print("I am here making dragonnet")
            net = DragonNet_transfer(x_s.shape[1],parm).to(device)
        
        # Train target data
        test_outputs, train_output, net, train_dict, test_dict = train_and_predict_dragons(
            t_t, y_t, x_t, net, seed=42, targeted_regularization=is_targeted_regularization,
            knob_loss=knob_loss, ratio=0.5, dragon=dragon, val_split=0.3, batch_size=batchsize2, lr=lr2, l1_reg=l1_reg)
        
        # Calculate errors
        for result_dict in [train_dict, test_dict]:
            truth = 2
            result_dict['index'] = result_dict['index'].tolist()
            result_dict['err'] = abs(truth - result_dict['psi_n']).mean()
            result_dict['dr_err'] = abs(truth - result_dict['dr_n']).mean()
            result_dict['ipw_error'] = abs(truth - result_dict['ipw_n']).mean()
        train_dict = {f'{k}_train': v.item() if 'index' not in k else v for k, v in train_dict.items()}
        test_dict = {f'{k}_test': v.item() if 'index' not in k else v for k, v in test_dict.items()}
        train_dict = {**train_dict,**test_dict}
        

        final_output.append(train_dict)
    
    # Save results
    serializable_output = convert_to_serializable(final_output)
    if not os.path.exists(f'./KMM-split-params/'):
        os.makedirs(f'./KMM-split-params/')
    with open(f'./KMM-split-params/KMM-split-experiments_transfer_{dragon}.json', 'w') as fp:
        json.dump(serializable_output, fp, indent=2)

In [ ]:
def turn_knob(data_base_dir='/Users/asus/Desktop/SCM/', knob='dragonnet',
              output_base_dir='',lr  = 1e-3, l1reg = 1e-4,batchsize = 64):
    output_dir = os.path.join(output_base_dir, knob)#扩充output_dir

    if knob == 'dragonnet':
        run_KMMsplit(data_base_dir=data_base_dir, output_dir=output_dir, dragon='dragonnet' ,lr2  = lr ,l1_reg = l1reg, batchsize2 = batchsize)

    if knob == 'tarnet':
        run_KMMsplit(data_base_dir=data_base_dir, output_dir=output_dir, dragon='tarnet',lr2  = lr ,l1_reg = l1reg, batchsize2 = batchsize)

In [ ]:
def main():
    parser = argparse.ArgumentParser()
    parser.add_argument('--data_base_dir', type=str, help="path to directory LBIDD", default="/Users/asus/Desktop/SCM")
    parser.add_argument('--knob', type=str, default='tarnet',
                        help="dragonnet or tarnet")

    parser.add_argument('--output_base_dir', type=str, help="directory to save the output",default="/Users/asus/Desktop/SCM")

    parser.add_argument('--transfer_lr',type = float,default=0.01)

    parser.add_argument('--l1reg',type = float,default=0.1)

    parser.add_argument('--batchsize',type = int,default=8)
    
    args, unknown = parser.parse_known_args()
    turn_knob(args.data_base_dir, args.knob, args.output_base_dir,args.transfer_lr, args.l1reg,args.batchsize)


if __name__ == '__main__':
    main()